# PyCity Schools Analysis

* As a whole, schools with higher budgets, did not yield better test results. By contrast, schools with higher spending per student actually (\$645-675) underperformed compared to schools with smaller budgets (<\$585 per student).

* As a whole, smaller and medium sized schools dramatically out-performed large sized schools on passing math performances (89-91% passing vs 67%).

* As a whole, charter schools out-performed the public district schools across all metrics. However, more analysis will be required to glean if the effect is due to school practices or the fact that charter schools tend to serve smaller student populations per school. 
---

### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import pandas as pd
import numpy as np

# File to Load (Remember to Change These)
school_data_to_load = "Resources/schools_complete.csv"
student_data_to_load = "Resources/students_complete.csv"

# Read School and Student Data File and store into Pandas Data Frames
school_data = pd.read_csv(school_data_to_load)
student_data = pd.read_csv(student_data_to_load)

# Combine the data into a single dataset
school_data_complete = pd.merge(student_data, school_data, how="left", on=["school_name", "school_name"])

## District Summary

* Calculate the total number of schools

* Calculate the total number of students

* Calculate the total budget

* Calculate the average math score 

* Calculate the average reading score

* Calculate the overall passing rate (overall average score), i.e. (avg. math score + avg. reading score)/2

* Calculate the percentage of students with a passing math score (70 or greater)

* Calculate the percentage of students with a passing reading score (70 or greater)

* Create a dataframe to hold the above results

* Optional: give the displayed data cleaner formatting

In [2]:
sa = str(len(school_data["school_name"].unique()))
sb = f'{student_data["student_name"].count():,}'
sc = f'${school_data["budget"].sum():,}'
sd = round(student_data["math_score"].mean(),2)
se = round(student_data["reading_score"].mean(),2)
sf = round((student_data["math_score"].mean()+student_data["reading_score"].mean())/2,2)
sg = f'{student_data[student_data["math_score"]>=70].count()["math_score"]/student_data["math_score"].count()*100:3.2f}%'     
sh = round(student_data[student_data["reading_score"]>=70].count()["reading_score"]/student_data["reading_score"].count()*100,2)     
    
# Creating a summary DataFrame using above values
summary_df = pd.DataFrame({"Total number of schools": [sa],
                           "Total number of students": [sb],
                           "Total budget": [sc],
                           "Average math score": [sd],
                           "Average reading score": [se],
                           "Average overall passing score": [sf],
                           "Percentage of students with a passing math score (70 or greater)": [sg],                           
                           "Percentage of students with a passing reading score (70 or greater)": str(sh)+"%"})
      
summary_df   

,Total number of schools,Total number of students,Total budget,Average math score,Average reading score,Average overall passing score,Percentage of students with a passing math score (70 or greater),Percentage of students with a passing reading score (70 or greater)
0,15,"39,170","$24,649,428",78.99,81.88,80.43,74.98%,85.81%


## School Summary

* Create an overview table that summarizes key metrics about each school, including:
  * School Name
  * School Type
  * Total Students
  * Total School Budget
  * Per Student Budget
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)
  
* Create a dataframe to hold the above results

## Top Performing Schools (By Passing Rate)

* Sort and display the top five schools in overall passing rate

In [3]:
# Combine the data into a single dataset
school_data_with_student = school_data_complete.groupby(["school_name"])
student_qty = school_data_with_student["school_name"].count()
school_type = school_data_with_student["type"].max()
school_budget = school_data_with_student["budget"].max()

per_student_budget = round(school_budget/student_qty,0)

avg_math_score = round(school_data_with_student["math_score"].sum()/school_data_with_student["school_name"].count(),2)
avg_reading_score = round(school_data_with_student["reading_score"].sum()/school_data_with_student["school_name"].count(),2)

passing_math = school_data_complete.loc[school_data_complete["math_score" ]>=70,["school_name","math_score"]]
per_passing_math = round(passing_math["school_name"].value_counts()/student_qty*100,2)

passing_reading = school_data_complete.loc[school_data_complete["reading_score" ]>=70,["school_name","reading_score"]]
per_passing_reading = round(passing_reading["school_name"].value_counts()/student_qty*100,2)

overall_passing_rate = round((per_passing_math + per_passing_reading)/2,2)

school_summary = pd.DataFrame({"student_qty" : student_qty, "school_type": school_type,
                               "school_budget" : school_budget, "per_student_budget": per_student_budget,
                                "avg_math_score" : avg_math_score, "avg_reading_score": avg_reading_score,
                                "%passing_math" : per_passing_math.map("{:.2f}%".format),
                                "%passing_reading" : per_passing_reading.map("{:.2f}%".format),
                                "%overall_passing_rate" :  overall_passing_rate.map("{:.2f}%".format)
                              })

# 5 best schools
school_summary_sorted1 = school_summary.sort_values("%overall_passing_rate", ascending=False)
best_school = school_summary_sorted1.head(5)
best_school

,student_qty,school_type,school_budget,per_student_budget,avg_math_score,avg_reading_score,%passing_math,%passing_reading,%overall_passing_rate
school_name,,,,,,,,,
Cabrera High School,1858,Charter,1081356,582.0,83.06,83.98,94.13%,97.04%,95.58%
Thomas High School,1635,Charter,1043130,638.0,83.42,83.85,93.27%,97.31%,95.29%
Pena High School,962,Charter,585858,609.0,83.84,84.04,94.59%,95.95%,95.27%
Griffin High School,1468,Charter,917500,625.0,83.35,83.82,93.39%,97.14%,95.26%
Wilson High School,2283,Charter,1319574,578.0,83.27,83.99,93.87%,96.54%,95.21%


## Bottom Performing Schools (By Passing Rate)

* Sort and display the five worst-performing schools

In [4]:
# 5 worst schools
school_summary_sorted = school_summary.sort_values("%overall_passing_rate", ascending=True)
worst_school = school_summary_sorted.head(5)
worst_school


,student_qty,school_type,school_budget,per_student_budget,avg_math_score,avg_reading_score,%passing_math,%passing_reading,%overall_passing_rate
school_name,,,,,,,,,
Rodriguez High School,3999,District,2547363,637.0,76.84,80.74,66.37%,80.22%,73.30%
Figueroa High School,2949,District,1884411,639.0,76.71,81.16,65.99%,80.74%,73.36%
Huang High School,2917,District,1910635,655.0,76.63,81.18,65.68%,81.32%,73.50%
Johnson High School,4761,District,3094650,650.0,77.07,80.97,66.06%,81.22%,73.64%
Ford High School,2739,District,1763916,644.0,77.10,80.75,68.31%,79.30%,73.81%


## Math Scores by Grade

* Create a table that lists the average Reading Score for students of each grade level (9th, 10th, 11th, 12th) at each school.

  * Create a pandas series for each grade. Hint: use a conditional statement.
  
  * Group each series by school
  
  * Combine the series into a dataframe
  
  * Optional: give the displayed data cleaner formatting

In [5]:
school_data_complete["grade"].value_counts()
math_9th = school_data_complete.loc[school_data_complete["grade" ]=="9th",["school_name","math_score"]]
math_9th_group = math_9th.groupby(["school_name"])
math_9th_list = round(math_9th_group["math_score"].mean(),2)

math_10th = school_data_complete.loc[school_data_complete["grade" ]=="10th",["school_name","math_score"]]
math_10th_group = math_10th.groupby(["school_name"])
math_10th_list = round(math_10th_group["math_score"].mean(),2)

math_11th = school_data_complete.loc[school_data_complete["grade" ]=="11th",["school_name","math_score"]]
math_11th_group = math_11th.groupby(["school_name"])
math_11th_list = round(math_11th_group["math_score"].mean(),2)

math_12th = school_data_complete.loc[school_data_complete["grade" ]=="12th",["school_name","math_score"]]
math_12th_group = math_12th.groupby(["school_name"])
math_12th_list = round(math_12th_group["math_score"].mean(),2)

school_math_avg_by_grade = pd.DataFrame({"9th" : math_9th_list,
                               "10th" : math_10th_list,
                                "11th" : math_11th_list,
                                "12th" : math_12th_list,
                              })
school_math_avg_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,77.08,77.00,77.52,76.49
Cabrera High School,83.09,83.15,82.77,83.28
Figueroa High School,76.40,76.54,76.88,77.15
Ford High School,77.36,77.67,76.92,76.18
Griffin High School,82.04,84.23,83.84,83.36
Hernandez High School,77.44,77.34,77.14,77.19
Holden High School,83.79,83.43,85.00,82.86
Huang High School,77.03,75.91,76.45,77.23
Johnson High School,77.19,76.69,77.49,76.86


## Reading Score by Grade 

* Perform the same operations as above for reading scores

In [6]:
school_data_complete["grade"].value_counts()
reading_9th = school_data_complete.loc[school_data_complete["grade" ]=="9th",["school_name","reading_score"]]
reading_9th_group = reading_9th.groupby(["school_name"])
reading_9th_list = round(reading_9th_group["reading_score"].mean(),2)

reading_10th = school_data_complete.loc[school_data_complete["grade" ]=="10th",["school_name","reading_score"]]
reading_10th_group = reading_10th.groupby(["school_name"])
reading_10th_list = round(reading_10th_group["reading_score"].mean(),2)

reading_11th = school_data_complete.loc[school_data_complete["grade" ]=="11th",["school_name","reading_score"]]
reading_11th_group = reading_11th.groupby(["school_name"])
reading_11th_list = round(reading_11th_group["reading_score"].mean(),2)

reading_12th = school_data_complete.loc[school_data_complete["grade" ]=="12th",["school_name","reading_score"]]
reading_12th_group = reading_12th.groupby(["school_name"])
reading_12th_list = round(reading_12th_group["reading_score"].mean(),2)

school_reading_avg_by_grade = pd.DataFrame({"9th" : reading_9th_list,
                               "10th" : reading_10th_list,
                                "11th" : reading_11th_list,
                                "12th" : reading_12th_list,
                              })
school_reading_avg_by_grade

,9th,10th,11th,12th
school_name,,,,
Bailey High School,81.30,80.91,80.95,80.91
Cabrera High School,83.68,84.25,83.79,84.29
Figueroa High School,81.20,81.41,80.64,81.38
Ford High School,80.63,81.26,80.40,80.66
Griffin High School,83.37,83.71,84.29,84.01
Hernandez High School,80.87,80.66,81.40,80.86
Holden High School,83.68,83.32,83.82,84.70
Huang High School,81.29,81.51,81.42,80.31
Johnson High School,81.26,80.77,80.62,81.23


## Scores by School Spending

* Create a table that breaks down school performances based on average Spending Ranges (Per Student). Use 4 reasonable bins to group school spending. Include in the table each of the following:
  * Average Math Score
  * Average Reading Score
  * % Passing Math
  * % Passing Reading
  * Overall Passing Rate (Average of the above two)

In [7]:
# Sample bins. Feel free to create your own bins.
spending_bins = [0, 585, 615, 645, 675]
group_names = ["<$585", "$585-615", "$615-645", "$645-675"]

In [8]:
#Scores by School Spending
budget_school_summary = school_summary.drop(["student_qty","school_type","school_budget"],axis=1)
budget_school_summary2 = budget_school_summary[["per_student_budget","avg_math_score","avg_reading_score",
                                               "%passing_math","%passing_reading", "%overall_passing_rate"]]
#delete column school_name
budget_school_summary2 = budget_school_summary2.reset_index()
budget_school_summary2 = budget_school_summary2.drop(["school_name"],axis=1)
budget_school_summary2

#display Spending Ranges (Per Student)
budget_school_summary2["Spending Ranges (Per Student)"] = pd.cut(budget_school_summary2["per_student_budget"], spending_bins, labels=group_names)
budget_school_summary2 = budget_school_summary2.groupby("Spending Ranges (Per Student)")
budgett = budget_school_summary2.mean()
budgett = budgett.drop(["per_student_budget"],axis=1)
budgett

,avg_math_score,avg_reading_score
Spending Ranges (Per Student),,
<$585,83.452500,83.935000
$585-615,83.600000,83.885000
$615-645,79.078333,81.891667
$645-675,76.996667,81.026667


## Scores by School Size

* Perform the same operations as above, based on school size.

In [9]:
# Sample bins. Feel free to create your own bins.
size_bins = [0, 1000, 2000, 5000]
group_names = ["Small (<1000)", "Medium (1000-2000)", "Large (2000-5000)"]

In [10]:
#Scores by School Size
size1 = school_summary.drop(["student_qty","school_type","school_budget","per_student_budget"],axis=1)
size2 = size1[["avg_math_score","avg_reading_score",
                "%passing_math","%passing_reading", "%overall_passing_rate"]]

size3 = pd.merge(size2,school_data, on="school_name", how="outer")
size3 = size3.drop(["School ID","budget","type", "school_name"],axis=1)

size3["School Size"] = pd.cut(size3["size"], size_bins, labels=group_names)
size_school = size3.drop(["size"], axis=1)
size_school = size_school.groupby("School Size")
size_school = round(size_school.mean(),2)
size_school 

,avg_math_score,avg_reading_score
School Size,,
Small (<1000),83.82,83.93
Medium (1000-2000),83.37,83.87
Large (2000-5000),77.74,81.34


## Scores by School Type

* Perform the same operations as above, based on school type.

In [24]:
# Sample bins. Feel free to create your own bins.
type_bins = [0,10,20]
group_names = ["Charter","District"]

#Scores by School Type
type1 = school_summary.drop(["student_qty","school_type","school_budget","per_student_budget"],axis=1)
type2 = type1[["avg_math_score","avg_reading_score",
                "%passing_math","%passing_reading", "%overall_passing_rate"]]

type3 = pd.merge(type2,school_data, on="school_name", how="outer")
type3 = type3.drop(["School ID","budget","size", "school_name"],axis=1)
type3["type_num"] = type3["type"].apply(lambda x: 2 if x == "Charter" else 12)
type3

type3["School Type"] = pd.cut(type3["type_num"], type_bins, labels=group_names)
type_school = type3.groupby("School Type")
type_school = round(type_school.mean(),2)
type_school 

,avg_math_score,avg_reading_score,type_num
School Type,,,
Charter,83.472500,83.897500,2
District,76.955714,80.965714,12
